# __ETL__ _(Extract, Transform, Load)_

## Introducción

Este notebook se enfoca en el proceso de **ETL** utilizando datos extraídos de las plataformas Yelp y Google Maps. Este proceso implica una _extracccion,transformación y carga_ de los datos con el objetivo de prepararlos para análisis posteriores. Este paso es crucial en cualquier proyecto de ciencia de datos para garantizar la calidad y utilidad de los datos.

## Configuraciones Globales e Importaciones

En esta sección, se instalan e importan todas las librerías y/o módulos necesarios para el proceso ETL (Extract, Transform, Load) y se establecen configuraciones globales de ser requerido. Se utilizan las siguientes librerías y herramientas:

In [35]:
import warnings
warnings.filterwarnings("ignore") # Se utiliza para gestionar las advertencias y mantener el código limpio.

In [36]:
import os # Proporciona funciones para interactuar con el sistema operativo.
import requests # Se utiliza para realizar solicitudes HTTP.
import pandas as pd # Una librería de análisis de datos.
import seaborn as sns #S e utiliza para la visualización de datos.
import pyspark.pandas as ps # Proporciona una interfaz para trabajar con datos en Spark utilizando el formato de DataFrame de pandas.
import json # Se utiliza para trabajar con datos en formato JSON.
from pyspark.sql import SparkSession # Se utiliza para crear una instancia de SparkSession, que es la entrada principal para trabajar con Spark SQL.
from pyspark.sql import functions as F #  Proporciona funciones para trabajar con datos en Spark DataFrame.
from pyspark.sql.functions import array_contains # Esta función se utiliza para filtrar los datos basados en la presencia de un valor en un array.
from pyspark.sql.functions import sum, col # Se utiliza para acceder a una columna en un DataFrame de Spark.
from pyspark.sql.functions import split, substring, concat_ws
from pyspark.sql.functions import expr

In [55]:
# Crear una sesión de Spark
spark = SparkSession.builder\
        .master("local")\
        .appName("ETL-reviews-estados")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# REVIEWS-ESTADOS

**Dataset:** NEW YORK y CALIFORNIA

DECLARACIÓN DE LA RUTA DE LOS DATA SET

In [39]:
# Ruta del directorio que contiene los archivos de reseñas de Nueva York
ruta_reviews_ny = "C:/Users/Usuario/Desktop/Proyecto Final/review-New_York"
ruta_reviews_cf = "C:/Users/Usuario/Desktop/Proyecto Final/reviews-California"

# Lista para almacenar los DataFrames de reseñas de Nueva York y California
df_NY = []
df_CA = []

# Inicialización de variables
i = 1
bandera = True

# Bucle para leer los archivos JSON de reseñas de Nueva York
while bandera:
    try:
        # Lectura del archivo JSON utilizando Spark y agregándolo a la lista
        archivo = spark.read.json(f"{ruta_reviews_ny}/{i}.json")
        df_NY.append(archivo)
        i += 1
    except:
        # Finalización del bucle si no hay más archivos
        bandera = False

# Inicialización de variables
i = 1
bandera = True

# Bucle para leer los archivos JSON de reseñas de California
while bandera:
    try:
        # Lectura del archivo JSON utilizando Spark y agregándolo a la lista
        archivo = spark.read.json(f"{ruta_reviews_cf}/{i}.json")
        df_CA.append(archivo)
        i += 1
    except:
        # Finalización del bucle si no hay más archivos
        bandera = False

# DataFrame final para almacenar las reseñas de Nueva York combinadas
df_final_reviews_ny = df_NY[0]
# Unión de los DataFrames de reseñas de Nueva York
for dataframe in df_NY[1:]:
    df_final_reviews_ny = df_final_reviews_ny.unionByName(dataframe)

# DataFrame final para almacenar las reseñas de California combinadas
df_final_reviews_ca = df_CA[0]
# Unión de los DataFrames de reseñas de California
for dataframe in df_CA[1:]:
    df_final_reviews_ca = df_final_reviews_ca.unionByName(dataframe)

# Sobrescribir df_NY y df_CA con los DataFrames finales combinados
df_NY = df_final_reviews_ny
df_CA = df_final_reviews_ca


In [40]:
# Concatenar los DataFrames 
df = df_NY.union(df_CA)

In [41]:
#Mostrar el DATAFRAME
df.show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|             gmap_id|                name|                pics|rating|                resp|                text|         time|             user_id|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|0x89c25fc9494dce4...|      Alvin Martinez|[{[https://lh5.go...|     5|                NULL|I'm late to posti...|1603494795361|11372210469230823...|
|0x89c25fc9494dce4...|     Johnnie Jackson|                NULL|     1|{We pride ourselv...|Very dissatisfied...|1620157037403|10729344149210932...|
|0x89c25fc9494dce4...|        Manie Blazer|                NULL|     5|                NULL|Excellent very we...|1597431662039|10037858580181940...|
|0x89c25fc9494dce4...|      Fashion Fiinds|                NULL|     5|{Thanks for the a...|Basing my revi

In [42]:
#Contamos filas
df.count()

5400000

In [43]:
# Cuenta el número de nulos en cada columna

def conteo_nulos(dataframe):
  conteo_nulos_por_columna = dataframe.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns])

  # Muestra el resultado
  conteo_nulos_por_columna.show()

In [46]:
conteo_nulos(df)

+-------+----+-------+------+-------+-------+----+-------+
|gmap_id|name|   pics|rating|   resp|   text|time|user_id|
+-------+----+-------+------+-------+-------+----+-------+
|      0|   0|5199054|     0|4890834|2334557|   0|      0|
+-------+----+-------+------+-------+-------+----+-------+



In [47]:
# Eliminar duplicados basándote en todas las columnas
df_no_duplicates = df.dropDuplicates()

# Contar las filas después de eliminar duplicados
print("Número de filas después de eliminar duplicados:", df_no_duplicates.count())

Número de filas después de eliminar duplicados: 5265421


Se procedio a buscar la cantidades de nulos que teniamos como tambien los duplicados. El _total de duplicados fue: 134,579_

En el siguiente paso procederemos a _eliminar las columnas pics y resp_ porque contienen más del 90% de los datos nulos

In [48]:
# Lista de columnas a eliminar 
columnas_a_eliminar = ['pics', 'resp'] 
# Elimina las columnas especificadas 
df = df.select([columna for columna in df.columns if columna not in columnas_a_eliminar])

In [50]:
df.show()

+--------------------+--------------------+------+--------------------+-------------+--------------------+
|             gmap_id|                name|rating|                text|         time|             user_id|
+--------------------+--------------------+------+--------------------+-------------+--------------------+
|0x89c25fc9494dce4...|      Alvin Martinez|     5|I'm late to posti...|1603494795361|11372210469230823...|
|0x89c25fc9494dce4...|     Johnnie Jackson|     1|Very dissatisfied...|1620157037403|10729344149210932...|
|0x89c25fc9494dce4...|        Manie Blazer|     5|Excellent very we...|1597431662039|10037858580181940...|
|0x89c25fc9494dce4...|      Fashion Fiinds|     5|Basing my review ...|1543773862044|11499816115301982...|
|0x89c25fc9494dce4...|      Andres Rieloff|     1|Bad! Disorganized...|1597279097718|11717818572842229...|
|0x89c25fc9494dce4...|   claribel placeres|     1|Worse customer ev...|1456098569126|11055512483166433...|
|0x89c25fc9494dce4...|        Manie B

In [51]:
# Transforma la columna "time" de milisegundos a timestamp.
df = df.withColumn("time", (col("time") / 1000).cast('timestamp'))

In [52]:
# Selecciona las columnas 'user_id' y 'name' para crear la tabla user. 
user = df.select('user_id', 'name').dropDuplicates()

In [53]:
user.show()

+--------------------+--------------------+
|             user_id|                name|
+--------------------+--------------------+
|11409232417054963...|         Nataly Diaz|
|11821652550697589...|          Miss Arrow|
|10447435380083412...|           Michael G|
|10076816597589615...|    FAITHFUL SaviOUR|
|11053794423377500...|Rafaelina. Rivera...|
|11226579810509041...|         Sam Smithen|
|10746510942947335...|    Shannon Kauderer|
|10566103281342686...|              Sal P.|
|11168740463737060...|      Matthew Hanson|
|11501759531064434...|       Amanda Rivera|
|11827956310180636...|            Daniel S|
|10133343740820331...|          Jae Soulja|
|11385457655859633...|       Julie Piracha|
|10348155874117546...|    Stephen Ferrigno|
|11146568800797908...|      Michelle Zhang|
|11792012147987443...|Colleen Kelly-Sle...|
|11799064012349328...|           Britt Liz|
|11359111095002785...|       Emily Karsten|
|10052668183086772...|   Sanjoy Purkaystha|
|11068973666000718...|        Pa

In [54]:
# Elimina la columna 'name' del DataFrame original en PySpark. 
df = df.drop('name')

## Carga de nuestro archivo

In [ ]:
# Ruta al archivo Parquet local 
file_path = 'C:/Users/Usuario/Desktop/Proyecto Final/PF_Google_yelp_Map/Notebook/reviews-estados-limpios.parquet' 
# Escribe el DataFrame a un archivo Parquet localmente 
df.write.parquet(file_path)